In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "food_CRM.settings")  # 👈 replace with your actual project name
django.setup()


d:\Data Scientist\my project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
from food_APP.models import OutsorceDBLead
import asyncio
from asgiref.sync import sync_to_async
from datetime import datetime

In [3]:
# Load the Excel sheet
df = pd.read_excel("C:/Users/HP/Desktop/Hogist/Data For DM.xlsx", sheet_name="sheet 6")

In [4]:
df.columns

Index(['SL NO', 'COMPANY NAME', 'Unnamed: 2', 'Unnamed: 3', 'PHONE', 'PHONE.1',
       'Unnamed: 6', 'EMAIL', 'ADDRESS', 'REMARKS', 'DATE', 'PROFILE SHARED'],
      dtype='object')

In [5]:
df = df.drop(index=33)
df = df.drop(index=2469)

In [6]:
# Standardize the remarks column
df["REMARKS"] = df["REMARKS"].astype(str).str.lower()

# Filter rows that match your keywords
filtered_df = df[df["REMARKS"].str.contains("rnr|not requirement|call me later", na=False)]

# Select N random rows (for example: 5)
#random_rows = filtered_df.sample(n=10)

# Show the result
print(filtered_df.tail(20))
df_1 = filtered_df.tail(20)

      SL NO                          COMPANY NAME Unnamed: 2 Unnamed: 3  \
2419   2420    Vimala Imports and Exports pvt ltd        NaN        NaN   
2420   2421     Manna Chemicals And Drugs Pvt Ltd        NaN        NaN   
2424   2425        Chennai Power Services Pvt Ltd        NaN        NaN   
2425   2426      Ethos Coatings Engineers Pvt Ltd        NaN        NaN   
2427   2428                 Paveres Infra Pvt Ltd        NaN        NaN   
2429   2430           Cityrene Builders Pvt. Ltd.        NaN        NaN   
2441   2442               JMA IT Solutions P Ltd.        NaN        NaN   
2450   2451                  Melss Automation Ltd        NaN        NaN   
2467   2468                  SDAS Trading Pvt Ltd        NaN        NaN   
2481   2482            Breeze Innovations Pvt Ltd        NaN        NaN   
2484   2485  Lucrative Corporate Solution Pvt Ltd        NaN        NaN   
2491   2492   Manna Chemicals And Drugs Pvt. Ltd.        NaN        NaN   
2504   2505       Zion Ne

In [7]:
# Clean and rename
df = df_1.rename(columns={
    "PHONE": "contact_number",
    "COMPANY NAME": "org_name",
    "EMAIL": "mail_id",
    "ADDRESS": "Address",
})

In [8]:
df = df[["org_name", "contact_number", "mail_id", "Address"]]
df = df[df["contact_number"].notnull()]
df["contact_number"] = df["contact_number"].astype(str).str.split(".").str[0]
df = df.tail(20)  # Just 15 rows

In [9]:
df

,org_name,contact_number,mail_id,Address
2419,Vimala Imports and Exports pvt ltd,9443239220,vimalaimportsandexports@gmail.com,C/9327th streetTNHB Mugappair Mugappair Chenna...
2420,Manna Chemicals And Drugs Pvt Ltd,9176787691,vino.0287@gmail.com,c/19-B Industrial Estate Mogapair West. Chenna...
2424,Chennai Power Services Pvt Ltd,9500198368,NaN,No.18 Ashirvatham Nagar Mogappair Chennai Tami...
2425,Ethos Coatings Engineers Pvt Ltd,9840823271,sales@ethosgroup.in,Near India Land Building Ambattur Mogappair Ja...
2427,Paveres Infra Pvt Ltd,9994516547,paversinfra@gmail.com,No.185 Ground Floor 12th Cross Street Nolambur...
2429,Cityrene Builders Pvt. Ltd.,9884400438,marketing@cityrene.com,- 6 W Mogappair Rd Mogappair Industrial Estate...
2441,JMA IT Solutions P Ltd.,9962354623,chacko@jma-it.com,1427 VOC Street Second Floor Thiruvalluvar Nag...
2450,Melss Automation Ltd,7708917688,ranjithirup@yahoo.co.in,2/344JJ nagar Chennai Tamil Nadu 600037
2467,SDAS Trading Pvt Ltd,9840764164,ash_ok2@yahoo.com,B-1 Ben Foundation Orthed Western Nelamar Scho...
2481,Breeze Innovations Pvt Ltd,9840296554,pmuruganchn@gmail.com,2/860Kalamegam SalaiMogappair West Chennai Tam...


In [12]:
df.columns


Index(['org_name', 'contact_number', 'mail_id', 'Address'], dtype='object')

In [11]:
df.to_excel("C:/Users/HP/Desktop/Hogist/test_DM.xlsx", index=False)

In [9]:
# Insert into DB
leads = [
    OutsorceDBLead(
        name=row["org_name"],
        contact_number=row["contact_number"],
        mail_id=row["mail_id"],
        Address=row["Address"],
        created_at=datetime.now(),
        source_come_from="PDataBase",
        status = "New"       
    )
    for row in df.to_dict(orient="records")
]

In [68]:
await sync_to_async(OutsorceDBLead.objects.bulk_create)(leads)
print("✅ Inserted from Jupyter Notebook")

d:\Data Scientist\my project\.venv\Lib\site-packages\django\db\models\fields\__init__.py:1416: RuntimeWarning: DateTimeField OutsorceDBLead.created_at received a naive datetime (2025-04-03 16:21:41.369414) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
d:\Data Scientist\my project\.venv\Lib\site-packages\django\db\models\fields\__init__.py:1416: RuntimeWarning: DateTimeField OutsorceDBLead.created_at received a naive datetime (2025-04-03 16:21:41.370451) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
d:\Data Scientist\my project\.venv\Lib\site-packages\django\db\models\fields\__init__.py:1416: RuntimeWarning: DateTimeField OutsorceDBLead.created_at received a naive datetime (2025-04-03 16:21:41.372446) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


✅ Inserted from Jupyter Notebook
